## IMPORTS

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import cv2
from tqdm import tqdm
from numpy.random import rand

import keras
from keras.optimizers import adam_v2
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras import initializers
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input, BatchNormalization, Reshape, Flatten, Conv2D, Conv2DTranspose

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint


## SETTINGS

In [1]:
IMG_SIZE = 128
data = []
labels = []

## Func for generation input data from source images

In [ ]:
def create_data():
    data_dir = './input_data'
    folder = 'images'
    
    path = os.path.join(data_dir, folder)
    
    for image in tqdm(os.listdir(path)[:1000]):
        temp_path = os.path.join(path, image)
        img_array = cv2.imread(os.path.join(path, image))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        label = temp_path.split(os.path.sep)[-2]
        labels.append(label)
        data.append(new_array)
#=============================
create_data()

## Looking for training images

In [ ]:
plt.imshow(data[10])
plt.show()

## Data Prepairing

In [ ]:
print(len(data))
data = np.array(data, dtype='float') / 127.5 - 1

In [ ]:
data.shape

## The function of creating and setting the discriminator model

In [2]:
def discriminator(in_shape=(128, 128, 3)):
    
    model = Sequential()
    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2D(512, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2D(1024, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2D(2045, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    opt = adam_v2.Adam(learning_rate=0.002, beta_1=0.5)
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
discriminator().summary()

## The function of creating and setting the generator model

In [ ]:
def generator(latent_dim):
    
    n_nodes = 2048 * 4 * 4
    
    model = Sequential()
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Reshape((4, 4, 2048)))
    model.add(Conv2DTranspose(1024, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Conv2DTranspose(3, (4,4), strides=(2,2), padding='same'))

    return model

In [ ]:
generator(100).summary()

## Functions for assembling the finished model and several additional modules

In [ ]:
def gan(g_model, d_model):
    
    d_model.trainable = False
    
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    
    opt = adam_v2.Adam(lr=0.002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

def real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    
    return X, y

def fake_samples(g_model, latent_dim, n_samples):
    x_input = latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    
    return X, y
    
def latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    
    return x_input

def save_plot(examples, epoch, n=4):
    examples = (examples + 1) / 2.0
    
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plpt.imshow((examples[i]*255).astype(np.uint8))
        
    filename = f'./gen_img/gen_num_{epoch+1}.png'
    plt.savefig(filename)
    plt.close()

## Preparing the Model for Training

In [ ]:
def summarize(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    X_real, y_real = real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    
    x_fake, y_fake = fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    
    print(f'>Real Accuracy: {acc_real*100} | Fake Accuracy: {acc_fake*100}')
    save_plot(x_fake, epoch)
    filename = f'./models/gen_model_num_{epoch+1}.h5'
    g_model.save(filename)

In [ ]:
def new_train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            X_fake, y_fake = fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            
            X_gan = latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            
        if (i + 1) % 10 == 0:
            summarize(i, g_model, d_model, dataset, latent_dim)
            

In [ ]:
def fs_for_training(n_samples):
    X = rand(128 * 128 * 3 * n_samples)
    X = X.reshape((n_samples, 128, 128, 3))
    y = zeros((n_samples, 1))
    
    return X, y

In [ ]:
def train_discriminator(d_model, dataset, n_iter=50, n_batch=256):
    half_batch = int(n_batch / 2)
    
    for i in range(n_iter):
        X_real, y_real = real_samples(dataset, half_batch)
        _, real_acc = d_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = fs_for_training(half_batch)
        _, fake_acc = d_model.train_on_batch(X_fake, y_fake)
        
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))
        
    return d_model
        

In [ ]:
latent_dim = 100
d_model = discriminator()
g_model = generator(latent_dim)

dataset = data
d_model = train_discriminator(d_model, dataset)

gan_model = gan(g_model, d_model)
gan_model.summary()

In [ ]:
new_train(g_model, d_model, gan_model, dataset, latent_dim)